In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11052\3761313380.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11052\3761313380.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
df = pd.read_csv("NioPracticeUserbase.csv")

In [3]:
df.head(10)

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment,Country,Age,Gender,Device,Plan Duration
0,1,Basic,10,1/15/2022 00:00,6/10/2023 00:00,United States,28,Male,Smartphone,1 Month
1,2,Premium,15,9/5/2021 00:00,6/22/2023 00:00,Canada,35,Female,Tablet,1 Month
2,3,Standard,12,2/28/2023 00:00,6/27/2023 00:00,United Kingdom,42,Male,Smart TV,1 Month
3,4,Standard,12,7/10/2022 00:00,6/26/2023 00:00,Australia,51,Female,Laptop,1 Month
4,5,Basic,10,5/1/2023 00:00,6/28/2023 00:00,Germany,33,Male,Smartphone,1 Month
5,6,Premium,15,3/18/2022 00:00,6/27/2023 00:00,France,29,Female,Smart TV,1 Month
6,7,Standard,12,12/9/2021 00:00,6/25/2023 00:00,Brazil,46,Male,Tablet,1 Month
7,8,Basic,10,4/2/2023 00:00,6/24/2023 00:00,Mexico,39,Female,Laptop,1 Month
8,9,Standard,12,10/20/2022 00:00,6/23/2023 00:00,Spain,37,Male,Smartphone,1 Month
9,10,Premium,15,1/7/2023 00:00,6/22/2023 00:00,Italy,44,Female,Smart TV,1 Month


In [4]:
df.describe()

,User ID,Monthly Revenue,Age
count,2500.00000,2500.000000,2500.000000
mean,1250.50000,12.508400,38.795600
std,721.83216,1.686851,7.171778
min,1.00000,10.000000,26.000000
25%,625.75000,11.000000,32.000000
50%,1250.50000,12.000000,39.000000
75%,1875.25000,14.000000,45.000000
max,2500.00000,15.000000,51.000000


In [5]:
df.describe(include=object)

,Subscription Type,Join Date,Last Payment,Country,Gender,Device,Plan Duration
count,2500,2500,2500,2500,2500,2500,2500
unique,3,300,26,10,2,4,1
top,Basic,11/5/2022 00:00,6/28/2023 00:00,United States,Female,Laptop,1 Month
freq,999,33,164,451,1257,636,2500


In [6]:
# Total Gender count by Country
df.groupby('Gender')['Country'].value_counts()

Gender  Country       
Female  Spain             233
        United States     225
        Canada            157
        Brazil             95
        Germany            94
        United Kingdom     93
        France             91
        Italy              91
        Australia          89
        Mexico             89
Male    United States     226
        Spain             218
        Canada            160
        Australia          94
        Mexico             94
        France             92
        Italy              92
        United Kingdom     90
        Germany            89
        Brazil             88
Name: Country, dtype: int64

In [7]:
subscription_count = df['Subscription Type'].value_counts()
print(subscription_count)

Basic       999
Standard    768
Premium     733
Name: Subscription Type, dtype: int64


In [8]:
# Subscription count by Country
subscription_by_country = df.groupby(['Country', 'Subscription Type']).size().unstack(fill_value=0)
print("Subscription counts by country:")
print(subscription_by_country)

Subscription counts by country:
Subscription Type  Basic  Premium  Standard
Country                                    
Australia             31      101        51
Brazil               146       33         4
Canada               145       88        84
France                36      147         0
Germany              149        3        31
Italy                176        4         3
Mexico                 4        0       179
Spain                110      212       129
United Kingdom         3        0       180
United States        199      145       107


In [9]:
# Subscription count by Gender
subscription_by_gender = df.groupby(['Gender', 'Subscription Type']).size().unstack(fill_value=0)
print("\nSubscription count by gender:")
print(subscription_by_gender)


Subscription count by gender:
Subscription Type  Basic  Premium  Standard
Gender                                     
Female               512      364       381
Male                 487      369       387


In [10]:
device_count = df.groupby('Device').size().reset_index(name='Device Count')
print("Device Count for Each Device:")
print(device_count)

Device Count for Each Device:
       Device  Device Count
0      Laptop           636
1    Smart TV           610
2  Smartphone           621
3      Tablet           633


In [11]:
device_count_gender = df.groupby(['Device', 'Gender']).size().reset_index(name='Device Count')
print("Device Count by Gender:")
print(device_count_gender)

Device Count by Gender:
       Device  Gender  Device Count
0      Laptop  Female           329
1      Laptop    Male           307
2    Smart TV  Female           305
3    Smart TV    Male           305
4  Smartphone  Female           300
5  Smartphone    Male           321
6      Tablet  Female           323
7      Tablet    Male           310


In [12]:
total_revenue_by_country = df.groupby('Country')['Monthly Revenue'].sum().reset_index(name='Total Revenue')
print("Total Revenue by Country:")
print(total_revenue_by_country)

Total Revenue by Country:
          Country  Total Revenue
0       Australia           2271
1          Brazil           2285
2          Canada           3950
3          France           2307
4         Germany           2260
5           Italy           2317
6          Mexico           2237
7           Spain           5662
8  United Kingdom           2318
9   United States           5664


In [13]:
#Converting join date columns into date time
if 'Join Date' in df.columns:
    df['Join Date'] = pd.to_datetime(df['Join Date'])
    df['YearMonth'] = df['Join Date'].dt.to_period("M").astype(str)


In [14]:
app = JupyterDash(__name__)

# Layout
app.layout = html.Div([
    html.H1("User Subscription Dashboard",style= {'text-align': 'center'}),

    # Demographics 
    html.Div([
        html.H2("User Demographics Insight", style = {'text-align':'left'}),
        dcc.Graph(id='gender-count-bar'),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in df['Country'].unique()],
            multi=True,
            value=df['Country'].unique(),
            placeholder='Select countries'
        )
    ]),

    # giving space between two sections
    html.Div(style={'height': '50px'}),

    # Subscription section
    html.Div([
        html.H2(" User Subscription Insight",style= {'text-align': 'center'}),
        dcc.Dropdown(
            id='duration-dropdown',
            options=[{'label': '1 Month', 'value': '1 Month'}],
            value='1 Month',
            style={'width': '50%'}
        ),
        dcc.Dropdown(
            id='gender-dropdown',
            options=[{'label': gender, 'value': gender} for gender in df['Gender'].unique()],
            multi=True,
            value=df['Gender'].unique(),
            placeholder='Select gender'
        ),
        dcc.Dropdown(
            id='region-dropdown',
            options=[{'label': region, 'value': region} for region in df['Country'].unique()],
            multi=True,
            value=df['Country'].unique(),
            placeholder='Select regions'
        ),
        dcc.Graph(id='subscription-pie-chart'),
    ]),

    #Revenue Section
    html.Div([
        html.H2("Revenue Over Time",style= {'text-align': 'center'}),
        dcc.Graph(id='monthly-revenue-line-chart'),
        dcc.Checklist(
            id='revenue-segmentation-checkbox',
            options=[
                {'label': 'Segment by Subscription Type', 'value': 'segmentation'},
            ],
            value=[],
            inline=True,
            style={'margin-left': '20px'}
        ),
    ]),

    # Charts for Total Device Count, Revenue by Device and Revenu by Country
    html.Div([
        html.H2(" Device Type & Revenue by Country",style= {'text-align': 'center'}),

       
        dcc.Dropdown(
            id='chart-type-dropdown',
            options=[
                {'label': 'Pie Chart ( Device  Type Proportion %)', 'value': 'pie'},
                {'label': 'Bar Chart (Revenue by Device Type)', 'value': 'bar'},
                {'label': 'Bar Chart (Revenue by Country)', 'value': 'revenue_by_country'},
            ],
            value='pie',
            style={'width': '50%'},
            placeholder='Select chart type'
        ),
        dcc.Graph(id='additional-chart'),
        html.Button('Toggle Gender', id='toggle-gender-button', n_clicks=0),
    ]),
])


C:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [15]:
# Callback for Demographics section
@app.callback(
    Output('gender-count-bar', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_gender_count_bar(selected_countries):
    filtered_df = df[df['Country'].isin(selected_countries)]

    total_counts = filtered_df['Gender'].value_counts().reset_index(name='Count')

    fig1 = px.bar(
        total_counts,
        x='index',
        y='Count',
        title='Total Number of Users by Gender',
        labels={'index': 'User Gender', 'Count': 'Number of Users'},
        color='index',
        barmode='group'
    )

    return fig1


In [16]:
# Callback for Subscription section
@app.callback(
    Output('subscription-pie-chart', 'figure'),
    [Input('duration-dropdown', 'value'),
     Input('gender-dropdown', 'value'),
     Input('region-dropdown', 'value')]
)
def update_pie_chart(selected_duration, selected_gender, selected_region):
    filtered_df = df[df['Plan Duration'] == selected_duration]
    
    if selected_gender:
        filtered_df = filtered_df[filtered_df['Gender'].isin(selected_gender)]

    if selected_region:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_region)]

    subscription_counts = filtered_df['Subscription Type'].value_counts().reset_index()

    fig = px.pie(
        subscription_counts,
        names='index',
        values='Subscription Type',
        title= 'Subscription Proportion %',
    )

    return fig


In [17]:
# Callback for Revenue Section
@app.callback(
    Output('monthly-revenue-line-chart', 'figure'),
    [Input('revenue-segmentation-checkbox', 'value')]
)
def update_line_chart(revenue_segmentation):
    if 'segmentation' in revenue_segmentation and not df.empty:
        subscription_revenue = df.groupby(['Subscription Type', 'YearMonth'])['Monthly Revenue'].sum().reset_index()
        fig = px.line(subscription_revenue, x='YearMonth', y='Monthly Revenue', color='Subscription Type',
                      title='Total Monthly Revenue Over Time (Segmented by Subscription Type)',
                      labels={'Monthly Revenue': 'Total Monthly Revenue'},
                      markers=True)
    else:
        total_revenue = df.groupby('YearMonth')['Monthly Revenue'].sum().reset_index()
        fig = px.line(total_revenue, x='YearMonth', y='Monthly Revenue',
                      title='Total Monthly Revenue Over Time',
                      labels={'Monthly Revenue': 'Total Monthly Revenue'},
                      markers=True)

    return fig


In [18]:
# Callback for Total Device Count, Revenue by Device and Revenu by Country
@app.callback(
    Output('additional-chart', 'figure'),
    [Input('chart-type-dropdown', 'value'),
     Input('toggle-gender-button', 'n_clicks')],
    [State('additional-chart', 'relayoutData')]
)
def update_additional_chart(chart_type, n_clicks, relayout_data):
    if chart_type == 'pie':
        total_device_counts = df['Device'].value_counts().reset_index()
        fig = px.pie(total_device_counts, names='index', values='Device', title='Total Device Count')
    elif chart_type == 'bar':
        device_counts_by_gender = df.groupby(['Device', 'Gender']).size().reset_index(name='Count')
        fig = px.bar(device_counts_by_gender, x='Device', y='Count', color='Gender', barmode='group',
                     title='Device Count by Gender')
    elif chart_type == 'revenue_by_country':
        revenue_by_country = df.groupby(['Country', 'Gender'])['Monthly Revenue'].sum().reset_index()
        if 'xaxis.range[0]' in relayout_data:
          
            fig = px.bar(revenue_by_country, x='Country', y='Monthly Revenue', color='Gender', barmode='group',
                         title='Revenue by Country', range_x=[relayout_data['xaxis.range[0]'], relayout_data['xaxis.range[1]']])
        else:
            fig = px.bar(revenue_by_country, x='Country', y='Monthly Revenue', color='Gender', barmode='group',
                         title='Revenue by Country')
    else:
        fig = px.line()

    return fig


In [19]:
# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
